In [1]:
%%time
import pandas as pd
data = pd.read_excel('card transactions.xlsx',index = False)

Wall time: 27.4 s


In [3]:
data.dtypes

Recnum                        int64
Cardnum                       int64
Date                 datetime64[ns]
Merchnum                     object
Merch description            object
Merch state                  object
Merch zip                   float64
Transtype                    object
Amount                      float64
Fraud                         int64
dtype: object

## 1. Data Preprocessing

In [30]:
##remove Fedex
data2 = data[data['Merch description'].str.contains('FEDEX', case=False) == False]

In [31]:
data2.shape

(84978, 10)

In [32]:
##keep only purchase type P
data3 = data2[data2['Transtype']=='P']

In [33]:
data3.shape

(84623, 10)

In [11]:
##fill merchnum with 00000

In [37]:
data3['Merchnum'].isna().sum()

3199

In [40]:
data3['Merchnum'].apply(str)

1          61003026333
2        4503082993600
7        6098208200062
8         602608969534
13       5725000466504
14       5725000466504
19        602608969284
23       4353000719908
26       5725000466504
28       5725000466504
31       4503738417400
32       7234000681204
37        602608969739
40       3443000643641
44       9765000409955
48        602608969138
49        602608038586
51       5000006000095
52             9050153
53       3397000747409
54       5000006000095
55       5000006000095
56       4800000627972
57        991808369338
58       5000006000095
59       5000006000095
60       5000006000095
61       5000006000095
62       5000006000095
63       3397000747409
             ...      
96722    4503082412500
96723     620000655290
96724    5859063805001
96725    6822408111449
96726     602608969534
96728    4503057341100
96729     465094667331
96730              nan
96731    7509000781541
96732     602608969138
96733    3500000004991
96734     990903450339
96735    57

In [41]:
data3["Merchnum"].fillna("00000", inplace = True) 

C:\Users\Wendy\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [42]:
data3['Merchnum'].isna().sum()

0

## 2.Take first digit

In [43]:
data3 = data3.reset_index().drop(columns = 'index')

In [48]:
data3['amount_first_digit'] = data3['Amount'].apply(lambda x: 100*x).apply(str).apply(lambda x: x[0]).apply(int)

In [49]:
data3['amount_first_digit'].unique()

array([3, 1, 2, 6, 4, 5, 7, 9, 8], dtype=int64)

## 3. Group by

In [50]:
merchnum_dis =pd.DataFrame(data3.groupby(['Merchnum','amount_first_digit']).count()['Recnum'].unstack(),columns = range(1,10)).fillna(0)

In [51]:
merchnum_dis['1-2'] = merchnum_dis[1]+merchnum_dis[2]
merchnum_dis['3-9'] = merchnum_dis[3]+merchnum_dis[4]+merchnum_dis[5]+merchnum_dis[6]+merchnum_dis[7]+merchnum_dis[8]+merchnum_dis[9]
merchnum_dis['1-2'] = merchnum_dis['1-2'].replace(0, 1)
merchnum_dis['3-9'] = merchnum_dis['3-9'].replace(0, 1)
merchnum_dis['R'] = 1.096* merchnum_dis['1-2']/merchnum_dis['3-9']
merchnum_dis['1/R'] = 1/merchnum_dis['R']
merchnum_dis['max'] = merchnum_dis[['R','1/R']].max(axis=1)
merchnum_dis['n'] = merchnum_dis['1-2']+merchnum_dis['3-9']
merchnum_dis['t'] = (merchnum_dis['n']-15)/3
merchnum_dis['U*'] = 1 + (merchnum_dis['max']-1)/(1+np.exp(-1*merchnum_dis['t']))
merchnum_dis.sort_values(by = 'U*',ascending = False).head(40)

,1,2,3,4,5,6,7,8,9,1-2,3-9,R,1/R,max,n,t,U*
Merchnum,,,,,,,,,,,,,,,,,
991808369338,0.0,0.0,181.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,181.0,0.006055,165.145985,165.145985,182.0,55.666667,165.145985
8078200641472,55.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,59.0,1.0,64.664000,0.015465,64.664000,60.0,15.000000,64.663981
308904389335,0.0,0.0,53.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,53.0,0.020679,48.357664,48.357664,54.0,13.000000,48.357557
3523000628102,33.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,1.0,37.264000,0.026836,37.264000,35.0,6.666667,37.217908
808998385332,1.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,36.0,0.030444,32.846715,32.846715,37.0,7.333333,32.825921
55158027,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,27.0,1.0,29.592000,0.033793,29.592000,28.0,4.333333,29.221627
8916500620062,0.0,0.0,0.0,12.0,0.0,19.0,0.0,0.0,0.0,1.0,31.0,0.035355,28.284672,28.284672,32.0,5.666667,28.190609
3910694900001,0.0,25.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,25.0,1.0,27.400000,0.036496,27.400000,26.0,3.666667,26.741995
881145544,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,26.304000,0.038017,26.304000,25.0,3.333333,25.432399


In [52]:
result = merchnum_dis.sort_values(by = 'U*',ascending = False).head(40)
result.to_csv('merchnum_result.csv')

In [53]:
cardnum_dis = pd.DataFrame(data3.groupby(['Cardnum','amount_first_digit']).count()['Recnum'].unstack(),columns = range(1,10)).fillna(0)

In [54]:
cardnum_dis['1-2'] = cardnum_dis[1]+cardnum_dis[2]
cardnum_dis['3-9'] = cardnum_dis[3]+cardnum_dis[4]+cardnum_dis[5]+cardnum_dis[6]+cardnum_dis[7]+cardnum_dis[8]+cardnum_dis[9]
cardnum_dis['1-2'] = cardnum_dis['1-2'].replace(0, 1)
cardnum_dis['3-9'] = cardnum_dis['3-9'].replace(0, 1)
cardnum_dis['R'] = 1.096 * cardnum_dis['1-2']/cardnum_dis['3-9']
cardnum_dis['1/R'] = 1/cardnum_dis['R']
cardnum_dis['U'] = cardnum_dis[['R','1/R']].max(axis=1)
cardnum_dis['n'] = cardnum_dis['1-2']+cardnum_dis['3-9']
cardnum_dis['t'] = (cardnum_dis['n']-15)/3
cardnum_dis['U*'] = 1 + (cardnum_dis['U']-1)/(1+np.exp(-1*cardnum_dis['t']))
cardnum_dis.sort_values(by = 'U*',ascending = False).head(40)

,1,2,3,4,5,6,7,8,9,1-2,3-9,R,1/R,U,n,t,U*
Cardnum,,,,,,,,,,,,,,,,,
5142253356,51.0,10.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,61.0,5.0,13.371200,0.074788,13.371200,66.0,17.000000,13.371199
5142299705,18.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,25.0,3.0,9.133333,0.109489,9.133333,28.0,4.333333,9.027976
5142197563,10.0,5.0,122.0,1.0,5.0,0.0,3.0,1.0,2.0,15.0,134.0,0.122687,8.150852,8.150852,149.0,44.666667,8.150852
5142194617,1.0,4.0,0.0,13.0,0.0,20.0,0.0,0.0,0.0,5.0,33.0,0.166061,6.021898,6.021898,38.0,7.666667,6.019548
5142288241,1.0,0.0,6.0,1.0,0.0,1.0,1.0,1.0,3.0,1.0,13.0,0.084308,11.861314,11.861314,14.0,-0.333333,5.533836
5142239140,4.0,12.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,16.0,3.0,5.845333,0.171077,5.845333,19.0,1.333333,4.834555
5142144931,3.0,3.0,15.0,10.0,1.0,3.0,0.0,1.0,0.0,6.0,30.0,0.219200,4.562044,4.562044,36.0,7.000000,4.558799
5142192606,6.0,7.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,13.0,2.0,7.124000,0.140371,7.124000,15.0,0.000000,4.062000
5142204384,124.0,75.0,12.0,11.0,8.0,5.0,10.0,5.0,3.0,199.0,54.0,4.038963,0.247588,4.038963,253.0,79.333333,4.038963


In [55]:
result1 = cardnum_dis.sort_values(by = 'U*',ascending = False).head(40)
result1.to_csv('cardnum_result.csv')